<a href="https://colab.research.google.com/github/sumanthd17/aspect-based-sentiment/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=4cedd6a46022ef6dfed4783b4dc84c904b2b7fd1f75e2a89d828a573b67cacda
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 160.6 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [4]:
!pip install transformers

     |████████████████████████████████| 778kB 5.4MB/s 
     |████████████████████████████████| 1.1MB 29.1MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
     |████████████████████████████████| 3.0MB 51.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c0c44cc88b85f44e940407488e2d2f41d9ef0e903db0d3f6ea70c02527a806c5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import time
import datetime
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')

import torch
import torch.nn.functional as F
import transformers as optimus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
!git clone https://github.com/sumanthd17/aspect-based-sentiment.git

Cloning into 'aspect-based-sentiment'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 84 (delta 40), reused 57 (delta 22), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [7]:
cd aspect-based-sentiment

/content/aspect-based-sentiment


In [8]:
def load_train_data(input_dir):
    df = pd.read_csv(input_dir + "train-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [9]:
def load_val_data(input_dir):
    df = pd.read_csv(input_dir + "val-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [10]:
def hyper_params():
    BATCH_SIZE = 32
    MAX_SEQ_LENGTH = 256
    LEARNING_RATE = 2e-5
    EPOCHS = 5
    WARMUP = 0.1
    return BATCH_SIZE, MAX_SEQ_LENGTH, LEARNING_RATE, WARMUP, EPOCHS

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
!python create_data.py

2977
747
1491


In [13]:
train_data = load_train_data('QA_pairs/')
val_data = load_val_data('QA_pairs/')

batch_size, max_seq_len, lr, warmup, epochs = hyper_params()
num_training_steps = int(len(train_data) / batch_size) * epochs
num_warmup_steps = warmup * num_training_steps

print(len(train_data))
print(len(val_data))

15008
3750


In [14]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=True)

In [15]:
sent2idx = {
    'None': 0,
    'Positive': 1,
    'Negative': 2
}

In [16]:
tokenizer('hi, my name is sumanth', 'I am an engineer')

{'input_ids': [101, 7632, 1010, 2026, 2171, 2003, 7680, 4630, 2232, 102, 1045, 2572, 2019, 3992, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
train = pd.DataFrame()

for i, row in train_data.iterrows():
  d = {}
  encoded = tokenizer(row['ques'], row['ans'])
  d['input_ids'] = encoded['input_ids']
  d['attention_mask'] = encoded['attention_mask']
  d['token_type_ids'] = encoded['token_type_ids']
  d['label'] = sent2idx[row['sentiment']]
  train = train.append(d, ignore_index=True)

train['input_ids'] = train['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['attention_mask'] = train['attention_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['token_type_ids'] = train['token_type_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [18]:
train

,attention_mask,input_ids,label,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3976...",2.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 6671...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 2236...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3808...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3976...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...,...,...
15003,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3808...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
15004,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3976...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
15005,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 6671...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15006,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 2236...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."


In [19]:
val = pd.DataFrame()

for i, row in val_data.iterrows():
  d = {}
  encoded = tokenizer(row['ques'], row['ans'])
  d['input_ids'] = encoded['input_ids']
  d['attention_mask'] = encoded['attention_mask']
  d['token_type_ids'] = encoded['token_type_ids']
  d['label'] = sent2idx[row['sentiment']]
  val = val.append(d, ignore_index=True)

val['input_ids'] = val['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['attention_mask'] = val['attention_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['token_type_ids'] = val['token_type_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [20]:
val

,attention_mask,input_ids,label,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 6671...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3808...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 2236...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3976...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 6671...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
3745,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3976...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
3746,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 2236...",1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
3747,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 6671...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3748,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2054, 2079, 2017, 2228, 2055, 1996, 3808...",0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."


In [21]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [22]:
input_ids = torch.tensor([r for r in train['input_ids']], dtype=torch.long)
attention_mask = torch.tensor([r for r in train['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([r for r in train['token_type_ids']], dtype=torch.long)
label_ids = torch.tensor([r for r in train['label']], dtype=torch.long)

In [23]:
train_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, label_ids)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [24]:
input_ids = torch.tensor([r for r in val['input_ids']], dtype=torch.long)
attention_mask = torch.tensor([r for r in val['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([r for r in val['token_type_ids']], dtype=torch.long)
label_ids = torch.tensor([r for r in val['label']], dtype=torch.long)

In [25]:
val_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, label_ids)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [26]:
model_class, pretrained_weights = (
    optimus.BertForSequenceClassification,
    "bert-base-uncased",
)

model = model_class.from_pretrained(pretrained_weights, num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [27]:
model = model.to(device)

In [28]:
no_decay = ['bias', 'gamma', 'beta']
optimizer_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
      ]

In [29]:
optimizer = optimus.AdamW(optimizer_parameters, lr=lr, correct_bias=False)
scheduler = optimus.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [30]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [31]:
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, 5):
    ## TRAINING
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 100 == 0 and not step == 0:
            elapsed = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        model.zero_grad()        

        loss, _ = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)

        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / (len(train_dataloader) * train_dataloader.batch_size)    
    
    training_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    ## VALIDATION

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0

    for step, batch in enumerate(tqdm(val_dataloader)):
        
        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        
        with torch.no_grad():        
            loss, logits = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)
            
        total_eval_loss += loss.item()

        logits = F.softmax(logits, dim=-1)
        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        outputs = np.argmax(logits, axis=1)

        total_eval_accuracy += np.sum(outputs == label_ids)        

    avg_val_accuracy = total_eval_accuracy / (len(val_dataloader) * val_dataloader.batch_size)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / (len(val_dataloader) * val_dataloader.batch_size)
    
    validation_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(str(datetime.timedelta(seconds=int(round(time.time() - total_t0))))))

  0%|          | 0/469 [00:00<?, ?it/s]

======== Epoch 1 / 5 ========
Training...


 21%|██▏       | 100/469 [01:14<04:35,  1.34it/s]

  Batch   100  of    469.    Elapsed: 0:01:15.


 43%|████▎     | 200/469 [02:29<03:20,  1.34it/s]

...

  Batch   400  of    469.    Elapsed: 0:04:59.


  0%|          | 0/118 [00:00<?, ?it/s]


  Average training loss: 0.00
  Training epcoh took: 0:05:50

Running Validation...


100%|██████████| 118/118 [00:28<00:00,  4.17it/s]

  Accuracy: 0.94
  Validation Loss: 0.01
  Validation took: 0:00:28

Training complete!
Total training took 0:31:32 (h:mm:ss)


In [49]:
## INFERENCE

def load_test_data(input_dir):
    df = pd.read_csv(input_dir + "test-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [50]:
test_data = load_test_data('QA_pairs/')

In [51]:
idx2sentiment = {
    0: "None",
    1: "Positive",
    2: "Negative"
}

In [52]:
all_aspects = ['price',
               'shopping',
               'transit-location', 
               'safety',
               'nightlife',
               'live',
               'multiculture',
               'green-nature',
               'touristy',
               'quiet',
               'dining',
               'general']

In [53]:
test_grouped_by_id = test_data.groupby(['id'])

In [54]:
model.eval()

final_preds = pd.DataFrame()
test_accuracy = 0

l = 0
for id, group in tqdm(test_grouped_by_id):

  test = pd.DataFrame()
  for i, row in group.iterrows():
    d = {}
    encoded = tokenizer(row['ques'], row['ans'])
    d['input_ids'] = encoded['input_ids']
    d['attention_mask'] = encoded['attention_mask']
    d['token_type_ids'] = encoded['token_type_ids']
    d['label'] = sent2idx[row['sentiment']]
    test = test.append(d, ignore_index=True)

  test['input_ids'] = test['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
  test['attention_mask'] = test['attention_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
  test['token_type_ids'] = test['token_type_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

  input_ids = torch.tensor([r for r in test['input_ids']], dtype=torch.long)
  attention_mask = torch.tensor([r for r in test['attention_mask']], dtype=torch.long)
  token_type_ids = torch.tensor([r for r in test['token_type_ids']], dtype=torch.long)
  label_ids = torch.tensor([r for r in test['label']], dtype=torch.long)

  input_ids = input_ids.to(device)
  attention_mask = attention_mask.to(device)
  token_type_ids = token_type_ids.to(device)
  label_ids = label_ids.to(device)

  with torch.no_grad():        
    loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=label_ids)

  logits = F.softmax(logits, dim=-1)
  logits = logits.detach().cpu().numpy()
  label_ids = label_ids.to('cpu').numpy()
  outputs = np.argmax(logits, axis=1)

  test_accuracy += np.sum(outputs == label_ids)
  l += len(group)

  test['pred'] = outputs

  res = [idx for idx, val in enumerate(outputs) if val != 0]
  for val in res:
    d = {}
    d['id'] = id
    d['text'] = group.iloc[val]['ans']
    d['aspect'] = group.iloc[val]['ques'].split(' ')[6]
    d['sentiment'] = idx2sentiment[outputs[val]]
    d['target'] = 'LOCATION1' if 'LOCATION1' in group.iloc[val]['ans'] else 'LOCATION2'
    final_preds = final_preds.append(d, ignore_index=True)

print(test_accuracy/len(test_grouped_by_id))

100%|██████████| 1491/1491 [05:17<00:00,  4.69it/s]

14.310529845741113


In [59]:
test_accuracy / l

0.9462084257206208

In [56]:
final_preds.to_csv('prediction.csv', index=False)

In [3]:
!nvidia-smi

Wed Jul 29 16:32:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  